# data enriched

In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [26]:
df = pd.read_csv('/home/young/liuyixin/dl_learning/material/dataset/covid19-global-forecasting-week-2/train.csv', sep=',')
df['Date'] = pd.to_datetime(df['Date'])
train_last_date = df.Date.unique()[-1]
print(f"Dataset has training data untill : {train_last_date}")

Dataset has training data untill : 2020-03-31T00:00:00.000000000


In [27]:
wpop = pd.read_csv('/home/young/liuyixin/dl_learning/material/dataset/worldpopulationbyage/WPP2019_PopulationByAgeSex_Medium.csv')

In [28]:
country_mapper = {
'Iran (Islamic Republic of)' : "Iran",
'Bolivia (Plurinational State of)' : 'Bolivia',
'Brunei Darussalam' : 'Brunei',
'Congo' : 'Congo (Kinshasa)',
'Democratic Republic of the Congo' : "Congo (Brazzaville)",
"Côte d'Ivoire": "Cote d'Ivoire",
"Gambia" : "Gambia, The",
"Republic of Korea": "Korea, South",
"Republic of Moldova": "Moldova",
'Réunion' : "Reunion",
'Russian Federation' : "Russia",
'China, Taiwan Province of China' : "Taiwan*",
"United Republic of Tanzania": "Tanzania",
"Bahamas": "The Bahamas",
"Gambia": "The Gambia",
"United States of America (and dependencies)" : "US",
"Venezuela (Bolivarian Republic of)" : "Venezuela",
'Viet Nam' : "Vietnam"}

In [29]:
def rename_countries(x, country_dict):
    new_name = country_dict.get(x)
    if new_name is not None:
        #print(x, "-->", new_name)
        return new_name
    else:
        return x

In [30]:
wpop

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0-4,0,5,630.044,661.578,1291.622
1,4,Afghanistan,2,Medium,1950,1950.5,5-9,5,5,516.206,487.335,1003.541
2,4,Afghanistan,2,Medium,1950,1950.5,10-14,10,5,461.378,423.326,884.704
3,4,Afghanistan,2,Medium,1950,1950.5,15-19,15,5,414.369,369.363,783.732
4,4,Afghanistan,2,Medium,1950,1950.5,20-24,20,5,374.109,318.392,692.501
...,...,...,...,...,...,...,...,...,...,...,...,...
1404748,716,Zimbabwe,2,Medium,2100,2100.5,80-84,80,5,265.750,419.683,685.433
1404749,716,Zimbabwe,2,Medium,2100,2100.5,85-89,85,5,127.913,248.914,376.827
1404750,716,Zimbabwe,2,Medium,2100,2100.5,90-94,90,5,30.277,78.801,109.078
1404751,716,Zimbabwe,2,Medium,2100,2100.5,95-99,95,5,3.581,13.659,17.240


In [31]:
wpop = wpop[wpop['Time']==2020].reset_index(drop=True)

In [32]:
wpop

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,2020,2020.5,0-4,0,5,2907.406,2765.103,5672.509
1,4,Afghanistan,2,Medium,2020,2020.5,5-9,5,5,2774.630,2641.593,5416.223
2,4,Afghanistan,2,Medium,2020,2020.5,10-14,10,5,2651.664,2540.450,5192.114
3,4,Afghanistan,2,Medium,2020,2020.5,15-19,15,5,2377.941,2251.504,4629.445
4,4,Afghanistan,2,Medium,2020,2020.5,20-24,20,5,2017.279,1909.206,3926.485
...,...,...,...,...,...,...,...,...,...,...,...,...
9298,716,Zimbabwe,2,Medium,2020,2020.5,80-84,80,5,13.034,30.854,43.888
9299,716,Zimbabwe,2,Medium,2020,2020.5,85-89,85,5,3.697,11.763,15.460
9300,716,Zimbabwe,2,Medium,2020,2020.5,90-94,90,5,0.540,2.487,3.027
9301,716,Zimbabwe,2,Medium,2020,2020.5,95-99,95,5,0.033,0.234,0.267


In [33]:
wpop['Location'] = wpop.Location.apply(lambda x : rename_countries(x, country_mapper))

In [34]:
clean_wpop = wpop[wpop['Location'].isin(df['Country_Region'].unique())].reset_index()

In [35]:
population_distribution = []
for country, gpdf in clean_wpop.groupby("Location"):
    aux = {f"age_{age_grp}": tot for age_grp, tot in zip(gpdf.AgeGrp, gpdf.PopTotal)}
    aux["Country_Region"] = country
    population_distribution.append(aux) 

In [36]:
df_pop_distrib = pd.DataFrame(population_distribution)

In [37]:
df_pop_distrib

,age_0-4,age_5-9,age_10-14,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,...,age_60-64,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+,Country_Region
0,5672.509,5416.223,5192.114,4629.445,3926.485,3109.386,2478.259,2056.387,1658.723,1304.736,...,612.638,444.858,312.451,168.004,75.605,24.460,5.192,0.629,0.039,Afghanistan
1,166.278,167.554,162.181,199.597,228.356,244.323,223.924,166.849,152.510,170.509,...,186.459,143.783,108.132,86.536,53.876,23.388,6.627,0.776,0.051,Albania
2,5041.518,4635.972,3821.402,2910.345,2999.837,3573.878,3727.719,3547.516,3014.029,2434.590,...,1387.342,1143.200,723.938,513.896,343.627,171.049,51.370,8.796,0.963,Algeria
3,5795.004,5104.164,4349.261,3532.518,2882.566,2423.471,2040.757,1658.972,1330.695,1082.040,...,481.968,303.929,212.513,119.216,59.937,19.878,4.259,0.480,0.038,Angola
4,7.355,7.228,6.814,7.053,7.579,7.315,7.066,7.028,6.916,6.944,...,4.658,3.385,2.501,1.600,0.923,0.510,0.182,0.035,0.004,Antigua and Barbuda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,3431.637,3282.087,2925.422,2535.432,2782.336,3164.651,3030.397,2546.798,2105.400,1825.100,...,1168.385,771.243,343.946,228.204,163.802,67.757,23.650,3.712,0.479,Uzbekistan
157,2363.409,2731.190,2657.796,2500.772,2101.628,2028.711,2064.324,1963.597,1867.720,1697.254,...,1177.441,865.114,603.581,386.279,226.565,118.750,49.076,14.875,2.878,Venezuela
158,7892.471,7586.101,7098.174,6500.869,6820.203,8569.293,8437.027,7763.897,7033.826,6539.136,...,4330.954,3011.733,1651.958,1126.514,860.586,609.173,274.625,98.225,23.850,Vietnam
159,2946.454,2717.088,2428.598,2121.583,1746.939,1433.444,1193.475,991.719,808.231,605.658,...,237.554,166.281,112.763,67.202,32.916,10.863,1.803,0.121,0.005,Zambia


In [38]:
# add missing countries with median values
no_data = []
for country in df['Country_Region'].unique():
    if country not in df_pop_distrib['Country_Region'].unique():
        aux = df_pop_distrib.drop('Country_Region', axis=1).median(axis=0).to_dict()
        aux["Country_Region"] = country
        no_data.append(aux)
df_no_data = pd.DataFrame(no_data)

df_pop_distrib = pd.concat([df_pop_distrib, df_no_data], axis=0)


In [39]:
df_pop_distrib

,age_0-4,age_5-9,age_10-14,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,...,age_60-64,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+,Country_Region
0,5672.509,5416.223,5192.114,4629.445,3926.485,3109.386,2478.259,2056.387,1658.723,1304.736,...,612.638,444.858,312.451,168.004,75.605,24.460,5.192,0.629,0.039,Afghanistan
1,166.278,167.554,162.181,199.597,228.356,244.323,223.924,166.849,152.510,170.509,...,186.459,143.783,108.132,86.536,53.876,23.388,6.627,0.776,0.051,Albania
2,5041.518,4635.972,3821.402,2910.345,2999.837,3573.878,3727.719,3547.516,3014.029,2434.590,...,1387.342,1143.200,723.938,513.896,343.627,171.049,51.370,8.796,0.963,Algeria
3,5795.004,5104.164,4349.261,3532.518,2882.566,2423.471,2040.757,1658.972,1330.695,1082.040,...,481.968,303.929,212.513,119.216,59.937,19.878,4.259,0.480,0.038,Angola
4,7.355,7.228,6.814,7.053,7.579,7.315,7.066,7.028,6.916,6.944,...,4.658,3.385,2.501,1.600,0.923,0.510,0.182,0.035,0.004,Antigua and Barbuda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,847.828,860.640,737.609,667.454,719.127,756.486,785.320,691.992,617.644,589.862,...,349.957,293.575,209.912,131.207,75.605,36.082,12.339,1.964,0.179,Liechtenstein
8,847.828,860.640,737.609,667.454,719.127,756.486,785.320,691.992,617.644,589.862,...,349.957,293.575,209.912,131.207,75.605,36.082,12.339,1.964,0.179,Monaco
9,847.828,860.640,737.609,667.454,719.127,756.486,785.320,691.992,617.644,589.862,...,349.957,293.575,209.912,131.207,75.605,36.082,12.339,1.964,0.179,Saint Kitts and Nevis
10,847.828,860.640,737.609,667.454,719.127,756.486,785.320,691.992,617.644,589.862,...,349.957,293.575,209.912,131.207,75.605,36.082,12.339,1.964,0.179,San Marino


In [40]:
# normalize features
norm_pop_distrib = df_pop_distrib.drop("Country_Region", axis=1).div(df_pop_distrib.drop("Country_Region", axis=1).sum(axis=1), axis=0)

In [41]:
norm_pop_distrib 

,age_0-4,age_5-9,age_10-14,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,...,age_55-59,age_60-64,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+
0,0.145717,0.139133,0.133376,0.118922,0.100864,0.079875,0.063662,0.052825,0.042610,0.033516,...,0.020752,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001
1,0.057780,0.058223,0.056356,0.069357,0.079351,0.084899,0.077811,0.057978,0.052995,0.059250,...,0.070160,0.064792,0.049963,0.037575,0.030070,0.018721,0.008127,0.002303,0.000270,0.000018
2,0.114969,0.105721,0.087145,0.066369,0.068410,0.081500,0.085009,0.080899,0.068733,0.055520,...,0.039638,0.031638,0.026070,0.016509,0.011719,0.007836,0.003901,0.001171,0.000201,0.000022
3,0.176321,0.155301,0.132332,0.107482,0.087706,0.073737,0.062093,0.050476,0.040488,0.032923,...,0.020610,0.014665,0.009247,0.006466,0.003627,0.001824,0.000605,0.000130,0.000015,0.000001
4,0.075106,0.073809,0.069582,0.072022,0.077394,0.074698,0.072155,0.071767,0.070623,0.070909,...,0.061606,0.047566,0.034566,0.025539,0.016339,0.009425,0.005208,0.001859,0.000357,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.042032,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019
8,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.042032,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019
9,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.042032,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019
10,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.042032,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019


In [42]:
norm_pop_distrib['total_pop'] = df_pop_distrib.drop("Country_Region", axis=1).sum(axis=1)

In [43]:
norm_pop_distrib

,age_0-4,age_5-9,age_10-14,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,...,age_60-64,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+,total_pop
0,0.145717,0.139133,0.133376,0.118922,0.100864,0.079875,0.063662,0.052825,0.042610,0.033516,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
1,0.057780,0.058223,0.056356,0.069357,0.079351,0.084899,0.077811,0.057978,0.052995,0.059250,...,0.064792,0.049963,0.037575,0.030070,0.018721,0.008127,0.002303,0.000270,0.000018,2877.800
2,0.114969,0.105721,0.087145,0.066369,0.068410,0.081500,0.085009,0.080899,0.068733,0.055520,...,0.031638,0.026070,0.016509,0.011719,0.007836,0.003901,0.001171,0.000201,0.000022,43851.043
3,0.176321,0.155301,0.132332,0.107482,0.087706,0.073737,0.062093,0.050476,0.040488,0.032923,...,0.014665,0.009247,0.006466,0.003627,0.001824,0.000605,0.000130,0.000015,0.000001,32866.268
4,0.075106,0.073809,0.069582,0.072022,0.077394,0.074698,0.072155,0.071767,0.070623,0.070909,...,0.047566,0.034566,0.025539,0.016339,0.009425,0.005208,0.001859,0.000357,0.000041,97.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019,9240.438
8,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019,9240.438
9,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019,9240.438
10,0.091752,0.093138,0.079824,0.072232,0.077824,0.081867,0.084987,0.074887,0.066841,0.063835,...,0.037872,0.031771,0.022717,0.014199,0.008182,0.003905,0.001335,0.000213,0.000019,9240.438


In [44]:
norm_pop_distrib["Country_Region"] = df_pop_distrib["Country_Region"]

In [45]:

del df_pop_distrib
del df_no_data
del clean_wpop
del wpop

df = df.merge(norm_pop_distrib, on="Country_Region", how='left')

In [46]:
df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,age_0-4,age_5-9,age_10-14,age_15-19,...,age_60-64,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+,total_pop
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.015738,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,29366,NaN,Zimbabwe,2020-03-27,5.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.016177,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927
20576,29367,NaN,Zimbabwe,2020-03-28,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.016177,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927
20577,29368,NaN,Zimbabwe,2020-03-29,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.016177,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927
20578,29369,NaN,Zimbabwe,2020-03-30,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.016177,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927


# somoker

In [48]:
# https://ourworldindata.org/smoking#prevalence-of-smoking-across-the-world
smokers = pd.read_csv('/home/young/liuyixin/dl_learning/material/dataset/smokingstats/share-of-adults-who-smoke.csv')

In [50]:

smokers = smokers[smokers.Year == 2016].reset_index(drop=True)

In [51]:
smokers_country_dict = {'North America' : "US",
 'Gambia' : "The Gambia",
 'Bahamas': "The Bahamas",
 "'South Korea'" : "Korea, South",
'Papua New Guinea' : "Guinea",
 "'Czech Republic'" : "Czechia",
 'Congo' : "Congo (Brazzaville)"}

In [52]:
smokers['Entity'] = smokers.Entity.apply(lambda x : rename_countries(x, smokers_country_dict))

In [53]:
smokers

,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,ALB,2016,28.700000
1,Algeria,DZA,2016,15.600000
2,Andorra,AND,2016,33.500000
3,Arab World,NaN,2016,21.954592
4,Argentina,ARG,2016,21.800000
...,...,...,...,...
181,Vietnam,VNM,2016,22.800000
182,World,OWID_WRL,2016,20.475517
183,Yemen,YEM,2016,18.400000
184,Zambia,ZMB,2016,13.800000


In [55]:
no_datas_smoker = []
for country in df['Country_Region'].unique():
    if country not in smokers.Entity.unique():
        mean_score = smokers[['Smoking prevalence, total (ages 15+) (% of adults)']].mean().to_dict()
        mean_score['Entity'] = country
        no_datas_smoker.append(mean_score)

In [59]:
no_data_smoker_df = pd.DataFrame(no_datas_smoker)   

In [63]:
smokers

,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,ALB,2016,28.700000
1,Algeria,DZA,2016,15.600000
2,Andorra,AND,2016,33.500000
3,Arab World,NaN,2016,21.954592
4,Argentina,ARG,2016,21.800000
...,...,...,...,...
181,Vietnam,VNM,2016,22.800000
182,World,OWID_WRL,2016,20.475517
183,Yemen,YEM,2016,18.400000
184,Zambia,ZMB,2016,13.800000


In [64]:
no_data_smoker_df

,"Smoking prevalence, total (ages 15+) (% of adults)",Entity
0,21.389448,Afghanistan
1,21.389448,Angola
2,21.389448,Antigua and Barbuda
3,21.389448,Bahamas
4,21.389448,Belize
5,21.389448,Bhutan
6,21.389448,Bolivia
7,21.389448,Cabo Verde
8,21.389448,Cameroon
9,21.389448,Central African Republic


In [61]:
clean_smoke_data = pd.concat([smokers, no_data_smoker_df], axis=0)[['Entity','Smoking prevalence, total (ages 15+) (% of adults)']]

In [65]:
clean_smoke_data

,Entity,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,28.700000
1,Algeria,15.600000
2,Andorra,33.500000
3,Arab World,21.954592
4,Argentina,21.800000
...,...,...
42,Syria,21.389448
43,Taiwan*,21.389448
44,Timor-Leste,21.389448
45,Trinidad and Tobago,21.389448


In [66]:
clean_smoke_data.rename(columns={"Entity": "Country_Region",
                                  "Smoking prevalence, total (ages 15+) (% of adults)" : "smokers_perc"}, inplace=True)

In [68]:
clean_smoke_data

,Country_Region,smokers_perc
0,Albania,28.700000
1,Algeria,15.600000
2,Andorra,33.500000
3,Arab World,21.954592
4,Argentina,21.800000
...,...,...
42,Syria,21.389448
43,Taiwan*,21.389448
44,Timor-Leste,21.389448
45,Trinidad and Tobago,21.389448


In [70]:
df = df.merge(clean_smoke_data, on="Country_Region", how='left')

In [71]:
df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,age_0-4,age_5-9,age_10-14,age_15-19,...,age_65-69,age_70-74,age_75-79,age_80-84,age_85-89,age_90-94,age_95-99,age_100+,total_pop,smokers_perc
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341,21.389448
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341,21.389448
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341,21.389448
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341,21.389448
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.011428,0.008026,0.004316,0.001942,0.000628,0.000133,0.000016,0.000001,38928.341,21.389448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,29366,NaN,Zimbabwe,2020-03-27,5.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927,15.800000
20576,29367,NaN,Zimbabwe,2020-03-28,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927,15.800000
20577,29368,NaN,Zimbabwe,2020-03-29,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927,15.800000
20578,29369,NaN,Zimbabwe,2020-03-30,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,0.012445,0.007714,0.005766,0.002953,0.001040,0.000204,0.000018,0.000001,14862.927,15.800000


In [72]:
def concat_country_province(country, province):
    if not isinstance(province, str):
        return country
    else:
        return country+"_"+province

# Concatenate region and province for training
df["Country_Region"] = df[["Country_Region", "Province_State"]].apply(lambda x : concat_country_province(x[0], x[1]), axis=1)

# Add Time Data information from Quarantine, Restrictions and Schools

In [86]:
# https://www.kaggle.com/koryto/countryinfo

country_info = pd.read_csv('/home/young/liuyixin/dl_learning/material/dataset/countryinfo/covid19countryinfo.csv')
country_info = country_info[~country_info.country.isnull()].reset_index(drop=True)

In [87]:
country_info.drop([ c for c in country_info.columns if c.startswith("Unnamed")], axis=1, inplace=True)

In [88]:
country_info.drop(columns=['pop', 'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus', 'medianage', "smokers", "sexratio"],
                  axis=1,
                  inplace=True)

In [90]:
# Columns with dates
country_info["quarantine"] = pd.to_datetime(country_info["quarantine"])
country_info["restrictions"] = pd.to_datetime(country_info["publicplace"])
country_info["schools"] = pd.to_datetime(country_info["schools"])

In [91]:
same_state = []
for country in df["Province_State"].unique():
    if country in country_info.country.unique():
        same_state.append(country)

In [93]:
country_to_state_country = {}
for state in same_state:
    #print(state)
    #print(df[df["Province/State"]==state]["Country/Region"].unique())
    #print("----")
    country_to_state_country[state] = df[df["Province_State"]==state]["Country_Region"].unique()[0]+"_"+state

country_info['country'] = country_info.country.apply(lambda x : rename_countries(x, country_to_state_country))

coutry_merge_info = country_info[["country", "density", "urbanpop", "hospibed", "lung", "femalelung", "malelung"]]

cols_median = ["density", "urbanpop", "hospibed", "lung", "femalelung", "malelung"]
coutry_merge_info.loc[:, cols_median] = coutry_merge_info.loc[:, cols_median].apply(lambda x: x.fillna(x.median()),axis=0)

In [99]:
merged = df.merge(coutry_merge_info, left_on="Country_Region", right_on="country", how="left")
merged.loc[:, cols_median] = merged.loc[:, cols_median].apply(lambda x: x.fillna(x.median()),axis=0)

country_dates_info = country_info[["country", "restrictions", "quarantine", "schools"]]

In [100]:
country_dates_info

,country,restrictions,quarantine,schools
0,Afghanistan,NaT,NaT,NaT
1,Albania,NaT,NaT,NaT
2,Algeria,NaT,NaT,NaT
3,Andorra,NaT,NaT,NaT
4,Antigua and Barbuda,NaT,NaT,NaT
...,...,...,...,...
244,US,2020-03-15,2020-03-23,2020-03-15
245,US,2020-03-15,NaT,2020-03-16
246,US,2020-03-17,2020-03-24,2020-03-17
247,US,2020-03-17,2020-03-25,2020-03-18


In [101]:
def update_dates(a_df, col_update):
    """
    This creates a boolean time series with one after the start of confinements (different types : schools, restrictions or quarantine)
    """
    gpdf = a_df.groupby("Country_Region")
    new_col = gpdf.apply(lambda df : df[col_update].notnull().cumsum()).reset_index(drop=True)
    a_df[col_update] = new_col

In [103]:
for col in ["restrictions", "quarantine", "schools"]:
    print(merged.shape)
    merged = merged.merge(country_dates_info[["country", col]],
                          left_on=["Country_Region", "Date"],
                          right_on=["country", col],
                          how="left",
                          )
    update_dates(merged, col)

(21560, 36)
(21560, 38)
(21560, 40)


In [105]:
merged

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,age_0-4,age_5-9,age_10-14,age_15-19,...,hospibed,lung,femalelung,malelung,country_y,restrictions,country_x,quarantine,country_y,schools
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.5,37.62,36.31,39.33,NaN,0,NaN,0,NaN,0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.5,37.62,36.31,39.33,NaN,0,NaN,0,NaN,0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.5,37.62,36.31,39.33,NaN,0,NaN,0,NaN,0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.5,37.62,36.31,39.33,NaN,0,NaN,0,NaN,0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0,0.145717,0.139133,0.133376,0.118922,...,0.5,37.62,36.31,39.33,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21555,29366,NaN,Zimbabwe,2020-03-27,5.0,1.0,0.141119,0.148740,0.129252,0.110267,...,1.7,22.53,19.31,26.47,NaN,0,NaN,0,NaN,0
21556,29367,NaN,Zimbabwe,2020-03-28,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,1.7,22.53,19.31,26.47,NaN,0,NaN,0,NaN,0
21557,29368,NaN,Zimbabwe,2020-03-29,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,1.7,22.53,19.31,26.47,NaN,0,NaN,0,NaN,0
21558,29369,NaN,Zimbabwe,2020-03-30,7.0,1.0,0.141119,0.148740,0.129252,0.110267,...,1.7,22.53,19.31,26.47,NaN,0,NaN,0,NaN,0


In [106]:
drop_country_cols = [x for x in merged.columns if x.startswith("country_")]
merged.drop(columns=drop_country_cols, axis=1, inplace=True)

In [108]:
merged.to_csv('enriched_covid_19.csv', index=None)